In [70]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [71]:
df = df['content'].drop_duplicates().dropna()
print(df.info(),'\n')
print(df.describe())

<class 'pandas.core.series.Series'>
Int64Index: 49888 entries, 0 to 49999
Series name: content
Non-Null Count  Dtype 
--------------  ----- 
49888 non-null  object
dtypes: object(1)
memory usage: 779.5+ KB
None 

count                                                 49888
unique                                                49888
top       WASHINGTON  —   Congressional Republicans have...
freq                                                      1
Name: content, dtype: object
